# Recommender Systems (Movie Reviews)
- Done as part of a SharpestMinds skills test
- Author: Chris Hodapp
- Date: 2018-02-04

This assumes that `ml-100k` from movielens data (should be the `ml-100k` archive from https://grouplens.org/datasets/movielens/100k/) has been downloaded and uncompressed in the local directory.

## TODOs

- Bipolar Slope One?
- Read Netflix post
- Maybe implement something easy like NMF?
- Finish blog post and link to it
- Visualization of latent factors
- Explain code a bit better (ditch the formal Python version)
- Put utility matrix someplace else since I only need it for Slope One

## Loading data

In [1]:
import pandas as pd
import numpy as np
import sklearn.model_selection

In [2]:
ml = pd.read_csv("ml-100k/u.data", sep="\t", header=None,
                 names=("user_id", "movie_id", "rating", "time"))
# Convert Unix seconds to a Pandas timestamp:
ml["time"] = pd.to_datetime(ml["time"], unit="s")

In [3]:
ml[:10]

,user_id,movie_id,rating,time
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16
5,298,474,4,1998-01-07 14:20:06
6,115,265,2,1997-12-03 17:51:28
7,253,465,5,1998-04-03 18:34:27
8,305,451,3,1998-02-01 09:20:17
9,6,86,3,1997-12-31 21:16:53


In [4]:
ml.shape

(100000, 4)

In [5]:
max_user  = int(ml["user_id"].max() + 1)
max_movie = int(ml["movie_id"].max() + 1)
max_user, max_movie, max_user * max_movie

(944, 1683, 1588752)

To get an idea of data sparsity:

In [6]:
ml.shape[0] / (max_user * max_movie)

0.06294248567429025

## Training/testing split:

In [7]:
ml_train, ml_test = sklearn.model_selection.train_test_split(ml, test_size=0.25)

## Conversion to utility matrix:

We need a mask for some later steps, hence the m > 0 step; ratings go only from 1 to 5, so values of 0 are automatically unknown/missing data.

In [8]:
def df2mat(df):
    m = np.zeros((max_user, max_movie))
    m[df["user_id"], df["movie_id"]] = df["rating"]
    return m, m > 0
ml_mat_train, ml_mask_train = df2mat(ml_train)
ml_mat_test,  ml_mask_test  = df2mat(ml_test)

If this were an actual large amount of data, which a 944x1683 matrix doesn't really count as, you'd probably want [sparse matrices](https://docs.scipy.org/doc/scipy/reference/sparse.html) and to use 8-bit ints rather than 32-bit floats, for instance:

```python
ml_mat = scipy.sparse.coo_matrix(
    (ml["rating"], (ml["user_id"], ml["movie_id"])),
    shape=(max_user, max_movie),
    dtype=np.int8)
```

## Slope One implementation

- Based on:  [Slope One Predictors for Online Rating-Based Collaborative Filtering](https://arxiv.org/pdf/cs/0702144v1.pdf)
- TODO: This needs better explanation but I'm not sure if it should reside here, in the Python code, or in the blag post

In [9]:
def deviation(M, mask):
    m,n = M.shape
    m2 = mask.astype(np.int)
    counts = m2.T @ m2
    S = m2.T @ M
    diffs = S.T - S
    dev = diffs / np.maximum(1, counts)
    return dev, counts

The implementation of 'deviation' above might be less-optimal vastly larger matrices. For one thing, Slope One doesn't really *need* a utility matrix, though it's easier from one. One could readily compute deviation from the list of ratings, though I don't know a fast way to do this.

In [10]:
def predict_one(M, mask, dev, counts, u, j, weighted = False):
    m,n = M.shape
    # S_u is a mask over M's columns for items user 'u' rated:
    S_u = mask[u, :]
    if weighted:
        # In 'Weighted Slope One', we sum over everything user 'u' rated,
        # regardless of whether other users rated both this and item j:
        S_u[j] = False
        c_j = counts[j, S_u]
        devs = dev[j, S_u]
        u = M[u, S_u]
        return ((devs + u) * c_j).sum() / max(1.0, c_j.sum())
    else:
        # In the 'Slope One' formula we are summing over R_j, which is:
        # Every item 'i' (i != j), such that: user 'u' rated item 'i', and
        # at least one other user rated both item 'i' and item 'j'.
        # Below we compute this likewise as a mask over M's columns:
        R_j = S_u * (counts[u, :] > 0)
        R_j[j] = False
        u = M[u, R_j].sum()
        devs = dev[j, R_j].sum()
        card = max(1.0, R_j.sum())
        return (u + devs) / card

In [11]:
def predict(M, mask, dev, counts, dataframe, weighted=False):
    err_mae = 0
    err_rms = 0
    for row in dataframe.itertuples():
        p = predict_one(M, mask, dev, counts,
                        row.user_id, row.movie_id, weighted=weighted)
        err_mae += np.abs(p - row.rating)
        err_rms += np.square(p - row.rating)
    err_mae = err_mae / len(dataframe)
    err_rms = np.sqrt(err_rms / len(dataframe))
    return err_mae, err_rms

In [12]:
# Compute deviation (which is basically our model) from training:
dev, counts = deviation(ml_mat_train, ml_mask_train)

In [13]:
err_mae_train, err_rms_train = predict(ml_mat_train, ml_mask_train, dev, counts, ml_train)
err_mae_test,  err_rms_test  = predict(ml_mat_test,  ml_mask_test,  dev, counts, ml_test)

In [14]:
print("Training error: MAE={}, RMS={}".format(err_mae_train, err_rms_train))
print("Testing error: MAE={}, RMS={}".format(err_mae_test, err_rms_test))

Training error: MAE=0.6655537410406179, RMS=0.8512228253964512
Testing error: MAE=0.7543173167845862, RMS=0.9626572451178984


## Weighted Slope One

In [15]:
err_mae_train, err_rms_train = predict(ml_mat_train, ml_mask_train, dev, counts, ml_train, True)
err_mae_test,  err_rms_test  = predict(ml_mat_test,  ml_mask_test,  dev, counts, ml_test,  True)
# why must I pass both dataframe and matrix?

In [16]:
print("Training error: MAE={}, RMS={}".format(err_mae_train, err_rms_train))
print("Testing error: MAE={}, RMS={}".format(err_mae_test, err_rms_test))

Training error: MAE=0.7383013392450227, RMS=0.9839407186975825
Testing error: MAE=0.8959770441268928, RMS=1.2432122955992078


## SVD implementation

This is based on [Matrix Factorization Techniques for Recommender Systems](http://ieeexplore.ieee.org/abstract/document/5197422/) (sorry, it's paywalled) which shows the same model in equation 5 but not the gradient. [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD) in Surprise uses the same equation, and gives the same formulas for gradient-descent.  I derive the gradient below just to verify it myself.

The prediction model is:

$$\hat{r}_{ui}=\mu + b_i + b_u + q_i^\top p_u$$

where $u$ is a user, $i$ is an item, $\mu$ is the overall average rating, $b_i$ and $b_u$ are per-item and per-user deviations respectively, and $q_i$ and $p_u$ are feature vectors.

The goal is to minimize $L_2$-regularized squared error:

$$E=\sum_{r_{ui} \in R_{\textrm{train}}} \left(r_{ui} - \hat{r}_{ui}\right)^2 + \lambda\left(b_i^2+b_u^2 + \lvert\lvert q_i\rvert\rvert^2 + \lvert\lvert p_u\rvert\rvert^2\right)$$

This is easily differentiable with respect to model parameters $b_i$, $b_u$, $q_i$, and $p_u$, so a normal approach is gradient-descent.  Finding gradient with respect to $b_i$ is straightforward:

$$
\begin{split}
\frac{\partial E}{\partial b_i} &= \sum_{r_{ui}} \frac{\partial}{\partial b_i} \left(r_{ui} - (\mu + b_i + b_u + q_i^\top p_u)\right)^2 + \frac{\partial}{\partial b_i}\lambda\left(b_i^2+b_u^2 + \lvert\lvert q_i\rvert\rvert^2 + \lvert\lvert p_u\rvert\rvert^2\right) \\
\frac{\partial E}{\partial b_i} &= \sum_{r_{ui}} 2\left(r_{ui} - (\mu + b_i + b_u + q_i^\top p_u)\right)(-1) + 2 \lambda b_i \\
\frac{\partial E}{\partial b_i} &= 2 \sum_{r_{ui}} \left(\lambda b_i + r_{ui} - \hat{r}_{ui}\right)
\end{split}
$$

Gradient with respect to $p_u$ proceeds similarly:

$$
\begin{split}
\frac{\partial E}{\partial p_u} &= \sum_{r_{ui}} \frac{\partial}{\partial p_u} \left(r_{ui} - (\mu + b_i + b_u + q_i^\top p_u)\right)^2 + \frac{\partial}{\partial p_u}\lambda\left(b_i^2+b_u^2 + \lvert\lvert q_i\rvert\rvert^2 + \lvert\lvert p_u\rvert\rvert^2\right) \\
\frac{\partial E}{\partial p_u} &= \sum_{r_{ui}} 2\left(r_{ui} - \hat{r}_{ui}\right)\left(-\frac{\partial}{\partial
p_u}q_i^\top p_u \right) + 2 \lambda p_u \\
\frac{\partial E}{\partial p_u} &= \sum_{r_{ui}} 2\left(r_{ui} - \hat{r}_{ui}\right)(-q_i^\top) + 2 \lambda p_u \\
\frac{\partial E}{\partial p_u} &= 2 \sum_{r_{ui}} \lambda p_u - \left(r_{ui} - \hat{r}_{ui}\right)q_i^\top
\end{split}
$$

Gradient with respect to $b_u$ is identical form to $b_i$, and gradient with respect to $q_i$ is identical form to $p_u$, except that the variables switch places.  The full gradients then have the standard form for gradient descent, i.e. a summation of a gradient term for each individual data point, so they turn easily into update rules for each parameter (which match the ones in the Surprise link) after absorbing the leading 2 into learning rate $\gamma$ and separating out the summation over each data point. That's given below, with $e_{ui}=r_{ui} - \hat{r}_{ui}$:

$$
\begin{split}
\frac{\partial E}{\partial b_i} &= 2 \sum_{r_{ui}} \left(\lambda b_i + e_{ui}\right)\ \ \ &\longrightarrow b_i' &= b_i - \gamma\frac{\partial E}{\partial b_i} &= b_i + \gamma\left(e_{ui} - \lambda b_u \right) \\
\frac{\partial E}{\partial b_u} &= 2 \sum_{r_{ui}} \left(\lambda b_u + e_{ui}\right)\ \ \ &\longrightarrow b_u' &= b_u - \gamma\frac{\partial E}{\partial b_u} &= b_u + \gamma\left(e_{ui} - \lambda b_i \right)\\
\frac{\partial E}{\partial p_u} &= 2 \sum_{r_{ui}} \lambda p_u - e_{ui}q_i^\top\ \ \ &\longrightarrow p_u' &= p_u - \gamma\frac{\partial E}{\partial p_u} &= p_u + \gamma\left(e_{ui}q_i - \lambda p_u \right) \\
\frac{\partial E}{\partial q_i} &= 2 \sum_{r_{ui}} \lambda q_i - e_{ui}p_u^\top\ \ \ &\longrightarrow q_i' &= q_i - \gamma\frac{\partial E}{\partial q_i} &= q_i + \gamma\left(e_{ui}p_u - \lambda q_i \right) \\
\end{split}
$$


In [17]:
# Hyperparameters (using Surprise defaults):
gamma = 0.005
lambda_ = 0.02
factors = 100
num_epochs = 20

In [18]:
# Average rating:
mu = ml["rating"].mean()
# Deviations per-item and per-user (initialized randomly):
b_i = np.zeros((max_movie,))
b_u = np.zeros((max_user,))
# Factor matrices:
q_i = np.random.randn(factors, max_movie) * 0.1
p_u = np.random.randn(factors, max_user) * 0.1

In [19]:
def svd_predict(items, users, b_i, b_u, q, p):
    """Returns prediction from the given arrays of items and users,
    on this SVD model.
    
    Parameters:
    items -- 1D array of item IDs
    users -- 1D array of user IDs (same length as :items:)
    b_i -- 1D array of per-item deviations, length N
    b_u -- 1D array of per-user deviations, length M
    q -- 2D factor matrix as a NumPy array, shape (F,M)
    p -- 2D factor matrix as a NumPy array, shape (F,N)
    """
    return mu + b_i[items] + b_u[users] + (q_i[:, items] * p_u[:, users]).sum(axis=0)

In [20]:
def svd_error(df, b_i, b_u, q, p):
    p = svd_predict(df.movie_id, df.user_id, b_i, b_u, q_i, p_u)
    rmse = np.sqrt(np.square(p - df.rating).sum() / len(df))
    mae = np.abs(p - df.rating).sum() / len(df)
    return rmse, mae

In [21]:
for epoch in range(num_epochs):
    idxs = np.random.permutation(len(ml_train))
    data = ml_train.iloc[idxs]
    for idx,row in enumerate(ml_train.itertuples()):
        u = row.user_id
        i = row.movie_id
        r_ui = row.rating
        e_ui = r_ui - svd_predict(i, u, b_i, b_u, q_i, p_u) # (mu + b_i[i] + b_u[u] + q_i[:, i].T @ p_u[:, u])
        dbi = gamma * (e_ui - lambda_ * b_u[u])
        dbu = gamma * (e_ui - lambda_ * b_i[i])
        dpu = gamma * (e_ui * q_i[:,i] - lambda_ * p_u[:, u])
        dqi = gamma * (e_ui * p_u[:,u] - lambda_ * q_i[:, i])
        b_i[i] += dbi
        b_u[u] += dbu
        p_u[:,u] += dpu
        q_i[:,i] += dqi
    train_rmse, train_mae = svd_error(ml_train, b_i, b_u, q_i, p_u)
    test_rmse, test_mae = svd_error(ml_test, b_i, b_u, q_i, p_u)
    print("Epoch {:02d}/{}; Training: MAE={:.3f} RMSE={:.3f}, Testing: MAE={:.3f} RMSE={:.3f}".format(epoch + 1, num_epochs, train_mae, train_rmse, test_mae, test_rmse))

Epoch 01/20; Training: MAE=0.804 RMSE=0.994, Testing: MAE=0.818 RMSE=1.012
Epoch 02/20; Training: MAE=0.762 RMSE=0.954, Testing: MAE=0.787 RMSE=0.985
Epoch 03/20; Training: MAE=0.741 RMSE=0.930, Testing: MAE=0.774 RMSE=0.973
Epoch 04/20; Training: MAE=0.726 RMSE=0.914, Testing: MAE=0.766 RMSE=0.966
Epoch 05/20; Training: MAE=0.714 RMSE=0.900, Testing: MAE=0.761 RMSE=0.961
Epoch 06/20; Training: MAE=0.704 RMSE=0.887, Testing: MAE=0.758 RMSE=0.958
Epoch 07/20; Training: MAE=0.694 RMSE=0.875, Testing: MAE=0.755 RMSE=0.955
Epoch 08/20; Training: MAE=0.685 RMSE=0.864, Testing: MAE=0.753 RMSE=0.953
Epoch 09/20; Training: MAE=0.676 RMSE=0.852, Testing: MAE=0.752 RMSE=0.952
Epoch 10/20; Training: MAE=0.666 RMSE=0.841, Testing: MAE=0.750 RMSE=0.950
Epoch 11/20; Training: MAE=0.657 RMSE=0.828, Testing: MAE=0.749 RMSE=0.949
Epoch 12/20; Training: MAE=0.646 RMSE=0.815, Testing: MAE=0.748 RMSE=0.948
Epoch 13/20; Training: MAE=0.636 RMSE=0.802, Testing: MAE=0.747 RMSE=0.947
Epoch 14/20; Training: MA

# Implementations in `scikit-surprise`

[Surprise](http://surpriselib.com/) contains implementations of many of the same things, so these are tested below. This same dataset is included as a built-in, but for consistency, we may as well load it from our dataframe.

In [22]:
import surprise
from surprise.dataset import Dataset

In [23]:
reader = surprise.Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ml[["user_id", "movie_id", "rating"]], reader)

In [24]:
surprise.model_selection.cross_validate(surprise.NormalPredictor(), data, cv=5)

{'fit_time': (0.10839986801147461,
  0.10835003852844238,
  0.1086585521697998,
  0.10732698440551758,
  0.10420656204223633),
 'test_mae': array([ 1.21711932,  1.22271572,  1.22381548,  1.21559752,  1.22134108]),
 'test_rmse': array([ 1.51721043,  1.52219415,  1.52415119,  1.51495726,  1.51840742]),
 'test_time': (0.23361635208129883,
  0.23522114753723145,
  0.23500919342041016,
  0.23406243324279785,
  0.2762880325317383)}

In [25]:
surprise.model_selection.cross_validate(surprise.SlopeOne(), data, cv=5)

{'fit_time': (0.7369599342346191,
  0.7611682415008545,
  0.948178768157959,
  1.1393687725067139,
  0.7258105278015137),
 'test_mae': array([ 0.74041847,  0.74460844,  0.74199455,  0.74515014,  0.74241609]),
 'test_rmse': array([ 0.94302116,  0.9465838 ,  0.94147874,  0.94827218,  0.94654138]),
 'test_time': (3.0613796710968018,
  2.5841076374053955,
  2.9052228927612305,
  3.0102553367614746,
  2.3535008430480957)}

In [26]:
surprise.model_selection.cross_validate(surprise.SVD(), data, cv=5)

{'fit_time': (5.512656211853027,
  5.4407196044921875,
  5.841166257858276,
  5.453616619110107,
  4.953552722930908),
 'test_mae': array([ 0.74090545,  0.74024858,  0.73754237,  0.73748592,  0.73526663]),
 'test_rmse': array([ 0.93790224,  0.93950221,  0.93363063,  0.94017359,  0.93339002]),
 'test_time': (0.30484724044799805,
  0.26099610328674316,
  0.2594015598297119,
  0.2566986083984375,
  0.2938072681427002)}